In [10]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [11]:
from tensorflow.keras import models, layers

In [15]:
Image_Size=256
Batch_Size=32

In [19]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "PatatoLeafDataSet",
    shuffle=True,
    image_size=(Image_Size,Image_Size),
    batch_size=Batch_Size
)

Found 2153 files belonging to 3 classes.


In [24]:
className=dataset.class_names
className

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']